In [1]:
import pandas as pd
import sqlite3


## Loading Orders Data (CSV)
This file contains transactional order-level information.


---



In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


## Loading Users Data (JSON)
This file contains user master data including city and membership.


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect("food_delivery.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)


## Loading Restaurants Data (SQL)
This simulates pulling data from a relational database.


In [5]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## Data Merging
Left joins are used to retain all order records.


In [6]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


Final Dataset

In [7]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


## Exploratory Analysis (Understanding the Dataset)


**1 . Order Trends Over Time**

This shows how order volume changes over time.

In [8]:
final_df.groupby("order_date")["order_id"].count().head()


,order_id
order_date,
01-01-2023,29
01-01-2024,27
01-02-2023,25
01-03-2023,37
01-04-2023,32


**2. User Behavior Patterns**

Compares order frequency between Gold and Regular users.

In [9]:
final_df.groupby("membership")["order_id"].count()


,order_id
membership,
Gold,4987
Regular,5013


**3. City-wise Performance**

Identifies top revenue-generating cities.



In [11]:
final_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)



,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Pune,1924797.93
Hyderabad,1889366.58


**4. Cuisine-wise Performance**

Shows revenue contribution by cuisine type.

In [12]:
final_df.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


**5. Membership Impact (Gold vs Regular)**

Highlights spending difference between membership types.

In [13]:
final_df.groupby("membership")["total_amount"].mean()


,total_amount
membership,
Gold,797.145556
Regular,805.158434


**6. Revenue Seasonality**

Helps analyze seasonal revenue patterns.

In [15]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["month"] = final_df["order_date"].dt.month

final_df.groupby("month")["total_amount"].sum()


,total_amount
month,
1,663020.26
2,630867.40
3,716738.98
4,666097.18
5,668428.61
6,610822.93
7,688559.45
8,694987.58
9,653838.07


MCQ code

In [16]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [17]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [20]:
user_total = final_df.groupby("user_id")["total_amount"].sum()
count_users = user_total[user_total > 1000].count()
count_users


np.int64(2544)

In [23]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-655451883.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [25]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [27]:
restaurant_count = final_df.groupby("cuisine")["restaurant_id"].nunique()
revenue = final_df.groupby("cuisine")["total_amount"].sum()

pd.concat([restaurant_count, revenue], axis=1)\
  .rename(columns={"restaurant_id": "restaurant_count", "total_amount": "revenue"})\
  .sort_values(["restaurant_count", "revenue"])


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


Chinese cuisine has the lowest number of distinct restaurants (120) while still generating substantial revenue compared to other cuisines.

In [37]:
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [39]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

filtered = restaurant_stats[
    restaurant_stats["total_orders"] < 20
].sort_values("avg_order_value", ascending=False)
filtered.head(10)

,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
Hotel Dhaba Chinese,973.125556,18
Amma Mess Pure Veg,965.299444,18
Hotel Biryani Pure Veg,964.577692,13
Annapurna Curry House Multicuisine,954.512353,17


In [40]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered.loc[filtered.index.isin(options)]


,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [41]:
final_df.groupby(["membership", "cuisine"])["total_amount"]\
        .sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [42]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [43]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [45]:
round(final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [46]:
final_df["user_id"].nunique()


2883

In [47]:
round(final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

In [48]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [49]:
top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]


1337

In [50]:
final_df.shape[0]

10000